# Assignment 1: Graphs Problems

**Mathematics for Robotics - Module 1: Graphs** \
*Master in Automation and Robotics - ETSII - UPM* \
Student: Josep María Barberá Civera


# Solving TSP, CPP and SPP

We are going to use `networkx` as the main library for the solution of this graph problems.

## Plotting Graphs

In [12]:
import networkx as nx
import os
import math
import glob
import time
import matplotlib.pyplot as plt
import numpy as np
graphs_dir = "/home/josep/Documents/ETSII/Matematicas_MUAR/1_Grafos/practica/Grafos"
os.chdir(graphs_dir)
tsp_files = glob.glob("*.tsp")
print(tsp_files)

['lin318.tsp', 'kroA100.tsp', 'pcb442.tsp']


In [2]:
# For ordered graphs
def get_pos(tsp_file):
    i = 0

    # we open and read each of the lines looking for the EOF
    with open(tsp_file , 'r') as this_file:
        lines = this_file.readlines()
        for line in lines:
            if line.find("EOF") != -1:
                i = lines.index(line)
                
    # Once we have the index line, we can read the previous one
    last_vertex_index = i-1
    last_vertex_line = lines[last_vertex_index]
    cut = last_vertex_line.find(" ")
    vertices_number = int(last_vertex_line[:cut])
    first_vertex_index = last_vertex_index - vertices_number + 1
    good_lines = lines[first_vertex_index:last_vertex_index+1]

    pos = {}
    for line in good_lines:
        l = line.find(" ")
        number = float(line[:l])
        line = line[l+1:]
        l = line.find(" ")
        x = float(line[:l])

        line = line[l+1:line.find("\n")]
        y = float(line)

        pos[number] = (x,y)
    return pos

In [3]:
def plot_graph(G, pos, filename):
    graph_dir = filename + '_plotgraph.png'
    fig = plt.figure(figsize=(30, 20), dpi=200)
    options = {
        'node_color':'red',
        'node_size':100,
        'alpha':1
    }
    nx.draw_networkx_nodes(G,pos=pos,**options)#draw nodes
    if len(pos) <= 100:
        nx.draw_networkx_edges(G,pos=pos,alpha=0.33,width=0.75)
    else:
        nx.draw_networkx_edges(G,pos=pos,alpha=0.033,width=0.5)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Add padding
    fig.savefig(graph_dir)

In [4]:
def plot_path(G, pos, path, plot_name):
    fig = plt.figure(figsize=(30, 20), dpi=200)
    H = G.copy()
    
    # Draw closest edges on each node only
    if len(pos) <= 150:
        nx.draw_networkx_edges(H, pos, edge_color="black", width=0.5, alpha=0.1)
    else:
        nx.draw_networkx_edges(H, pos, edge_color="black", width=0.5, alpha=0.01)
    # Draw the route
    nx.draw_networkx(
        G,
        pos,
        node_color = "midnightblue",
        font_color="whitesmoke",
        with_labels=True,
        edgelist=path,
        edge_color="red",
        node_size=400,
        width=3,
    )
    
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Add padding
    plot_name = plot_name + ".png"
    fig.savefig(plot_name)

In [5]:
def create_video_frames(G, pos, list_path, video_name):
    # Path 
    new_directory = "frames_" + video_name
    path = os.path.join(graphs_dir, new_directory) 
    os.mkdir(path)
    os.chdir(path)
    edge_list = []
    j = 0
    new_pos = {}
    for i in list_path:
        print(i)
        j = j + 1
        edge_list.append(i)
        u, v = i
        new_pos[u] = pos[u]
        new_pos[v] = pos[v]
        fig = plt.figure(figsize=(24, 16), dpi=200)
        H = G.copy()
    
        # Draw closest edges on each node only
        if len(pos) <= 150:
            nx.draw_networkx_edges(H, pos, edge_color="black", width=0.25, alpha=0.1)
        else:
            nx.draw_networkx_edges(H, pos, edge_color="black", width=0.25, alpha=0.01)
        # Draw the route
        nx.draw_networkx(
            G,
            pos,
            node_color = "midnightblue",
            font_color="whitesmoke",
            font_size = 9,
            with_labels=True,
            edgelist=edge_list,
            edge_color="red",
            node_size=100,
            width=2,
        )
        
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Add padding
        plot_name = str(j) + ".png"
        fig.savefig(plot_name)
        if j == 31:
            break
    os.chdir(graphs_dir)

In [6]:
#pos = get_pos(tsp_files[2])
#
#G = nx.complete_graph(range(1, len(pos)+1))
#nx.set_node_attributes(G,pos,'pos')
#plot_graph(G,pos,"testing")

## Some useful functions

In [7]:
def distance(pos, u, v):
    dx = pos[u][0] - pos[v][0]
    dy = pos[u][1] - pos[v][1]
    return math.sqrt(dx*dx + dy*dy)

In [8]:
def compute_length(G, path):
    length = 0
    j = 0
    for i in path:
        if j == 0:
            j = i
            continue
        else:
            length = length + G.get_edge_data(j,i)["weight"]
            j = i
    return length

## Main function for computing TSP, CPP and Shortest Path

In [10]:
os.chdir(graphs_dir)
for file in tsp_files:
    pos = get_pos(file)
    # Creating complete graph
    G = nx.complete_graph(range(1, len(pos)+1))
    nx.set_node_attributes(G,pos,'pos')
    
    # Add weights as Euclidean distance
    total_d = 0
    for u,v in G.edges():
        d = distance(pos, u, v)
        G.add_edge(u,v,weight = d)
        total_d = total_d + d
    
    # Here you decide which problem to solve
    ################ 1 => TSP: Traveling Salesman Problem
    GRAPH = 2 ###### 2 => CPP: chinese Postman Problem
    ################ 3 => SPP: Shortest Path Problem
    
    start_time = time.time()
    if GRAPH == 1:
        ##############################################
        # TSP (Travellin Salesman Problem)
        ##############################################
        tsp = nx.approximation.traveling_salesman_problem
        path = tsp(G)
        plot_name = "tsp_" + file[:-4] + ".png"
    elif GRAPH == 2:
        ##############################################
        # CPP or EULERIAN CIRCUIT = CPP (Chinese Postman Problem) (becuase we use full connected graphs)
        ##############################################
        if nx.is_eulerian(G):
               path = [u for u, v in nx.eulerian_circuit(G, source=1)]
        else:
            H = nx.eulerize(G)
            path = [u for u, v in nx.eulerian_path(H, source=1)]
        plot_name = "cpp_" + file[:-4] + ".png"
    elif GRAPH == 3:
        ##############################################
        # SHORTEST PATH
        ##############################################
        path = nx.shortest_path(G, source=1, target=len(pos))
        plot_name = "shortest_path_" + file[:-4] + ".png"
        ##############################################
        
    end_time = time.time()
    
    # Compute the total length for the path
    length = compute_length(G, path)
    
    print(f"Graph: {file}")
    print(f"Graph problem took {end_time - start_time:.4f} seconds")
    print(f"Here the path:\n{path}")
    print(f"With a total length of {length:.2f} distance units")
    print(f"With a total distance of graph: {total_d}\n")
    
    ## Uncomment for plotting the path
    #edge_list = list(nx.utils.pairwise(path))
    #plot_path(G, pos, edge_list, plot_name)
    
    ## Uncomment for creating video frames: really time consumming... 
    #create_video_frames(G, pos, edge_list, plot_name[:-4])

Graph: lin318.tsp
Graph problem took 13.4672 seconds
Here the path:
[1, 2, 3, 1, 4, 2, 5, 1, 6, 2, 7, 1, 8, 2, 9, 1, 10, 2, 11, 1, 12, 2, 13, 1, 14, 2, 15, 1, 16, 2, 17, 1, 18, 2, 19, 1, 20, 2, 21, 1, 22, 2, 23, 1, 24, 2, 25, 1, 26, 2, 27, 1, 28, 2, 29, 1, 30, 2, 31, 1, 32, 2, 33, 1, 34, 2, 35, 1, 36, 2, 37, 1, 38, 2, 39, 1, 40, 2, 41, 1, 42, 2, 43, 1, 44, 2, 45, 1, 46, 2, 47, 1, 48, 2, 49, 1, 50, 2, 51, 1, 52, 2, 53, 1, 54, 2, 55, 1, 56, 2, 57, 1, 58, 2, 59, 1, 60, 2, 61, 1, 62, 2, 63, 1, 64, 2, 65, 1, 66, 2, 67, 1, 68, 2, 69, 1, 70, 2, 71, 1, 72, 2, 73, 1, 74, 2, 75, 1, 76, 2, 77, 1, 78, 2, 79, 1, 80, 2, 81, 1, 82, 2, 83, 1, 84, 2, 85, 1, 86, 2, 87, 1, 88, 2, 89, 1, 90, 2, 91, 1, 92, 2, 93, 1, 94, 2, 95, 1, 96, 2, 97, 1, 98, 2, 99, 1, 100, 2, 101, 1, 102, 2, 103, 1, 104, 2, 105, 1, 106, 2, 107, 1, 108, 2, 109, 1, 110, 2, 111, 1, 112, 2, 113, 1, 114, 2, 115, 1, 116, 2, 117, 1, 118, 2, 119, 1, 120, 2, 121, 1, 122, 2, 123, 1, 124, 2, 125, 1, 126, 2, 127, 1, 128, 2, 129, 1, 130, 2, 131, 

Graph: kroA100.tsp
Graph problem took 0.9095 seconds
Here the path:
[1, 2, 3, 1, 4, 2, 5, 1, 6, 2, 7, 1, 8, 2, 9, 1, 10, 2, 11, 1, 12, 2, 13, 1, 14, 2, 15, 1, 16, 2, 17, 1, 18, 2, 19, 1, 20, 2, 21, 1, 22, 2, 23, 1, 24, 2, 25, 1, 26, 2, 27, 1, 28, 2, 29, 1, 30, 2, 31, 1, 32, 2, 33, 1, 34, 2, 35, 1, 36, 2, 37, 1, 38, 2, 39, 1, 40, 2, 41, 1, 42, 2, 43, 1, 44, 2, 45, 1, 46, 2, 47, 1, 48, 2, 49, 1, 50, 2, 51, 1, 52, 2, 53, 1, 54, 2, 55, 1, 56, 2, 57, 1, 58, 2, 59, 1, 60, 2, 61, 1, 62, 2, 63, 1, 64, 2, 65, 1, 66, 2, 67, 1, 68, 2, 69, 1, 70, 2, 71, 1, 72, 2, 73, 1, 74, 2, 75, 1, 76, 2, 77, 1, 78, 2, 79, 1, 80, 2, 81, 1, 82, 2, 83, 1, 84, 2, 85, 1, 86, 2, 87, 1, 88, 2, 89, 1, 90, 2, 91, 1, 92, 2, 93, 1, 94, 2, 95, 1, 96, 2, 97, 1, 98, 2, 99, 1, 100, 2, 99, 3, 4, 5, 3, 6, 4, 7, 3, 8, 4, 9, 3, 10, 4, 11, 3, 12, 4, 13, 3, 14, 4, 15, 3, 16, 4, 17, 3, 18, 4, 19, 3, 20, 4, 21, 3, 22, 4, 23, 3, 24, 4, 25, 3, 26, 4, 27, 3, 28, 4, 29, 3, 30, 4, 31, 3, 32, 4, 33, 3, 34, 4, 35, 3, 36, 4, 37, 3, 38, 4, 39

Graph: pcb442.tsp
Graph problem took 50.1948 seconds
Here the path:
[1, 2, 3, 1, 4, 2, 5, 1, 6, 2, 7, 1, 8, 2, 9, 1, 10, 2, 11, 1, 12, 2, 13, 1, 14, 2, 15, 1, 16, 2, 17, 1, 18, 2, 19, 1, 20, 2, 21, 1, 22, 2, 23, 1, 24, 2, 25, 1, 26, 2, 27, 1, 28, 2, 29, 1, 30, 2, 31, 1, 32, 2, 33, 1, 34, 2, 35, 1, 36, 2, 37, 1, 38, 2, 39, 1, 40, 2, 41, 1, 42, 2, 43, 1, 44, 2, 45, 1, 46, 2, 47, 1, 48, 2, 49, 1, 50, 2, 51, 1, 52, 2, 53, 1, 54, 2, 55, 1, 56, 2, 57, 1, 58, 2, 59, 1, 60, 2, 61, 1, 62, 2, 63, 1, 64, 2, 65, 1, 66, 2, 67, 1, 68, 2, 69, 1, 70, 2, 71, 1, 72, 2, 73, 1, 74, 2, 75, 1, 76, 2, 77, 1, 78, 2, 79, 1, 80, 2, 81, 1, 82, 2, 83, 1, 84, 2, 85, 1, 86, 2, 87, 1, 88, 2, 89, 1, 90, 2, 91, 1, 92, 2, 93, 1, 94, 2, 95, 1, 96, 2, 97, 1, 98, 2, 99, 1, 100, 2, 101, 1, 102, 2, 103, 1, 104, 2, 105, 1, 106, 2, 107, 1, 108, 2, 109, 1, 110, 2, 111, 1, 112, 2, 113, 1, 114, 2, 115, 1, 116, 2, 117, 1, 118, 2, 119, 1, 120, 2, 121, 1, 122, 2, 123, 1, 124, 2, 125, 1, 126, 2, 127, 1, 128, 2, 129, 1, 130, 2, 131, 

# Solving MST
MST: Minimum Spanning Tree

In [ ]:
import networkx as nx
import os
import math
import glob
import time
import matplotlib.pyplot as plt
import numpy as np
graphs_dir = "/home/josep/Documents/ETSII/Matematicas_MUAR/1_Grafos/practica/Grafos"
os.chdir(graphs_dir)
tsp_files = glob.glob("*.txt")
print(tsp_files)

In [ ]:
def create_graph(txt_file):
    i = 0
    # We create and empty graph
    G = nx.Graph()
    # we open and read each of the files looking for the EOF
    with open(txt_file , 'r') as this_file:
        lines = this_file.readlines()
        lines = lines[1:]
        lines.sort()
        for line in lines:
            if line.find("EOF") != -1:
                continue
            l = line.find(" ")
            u = int(line[:l])
            line = line[l+1:]
            l = line.find(" ")
            v = int(line[:l])
            line = line[l+1:line.find("\n")]
            w = float(line)
            
            # and for each line we add an edge
            G.add_edge(u, v, weight = w)
    return G

In [ ]:
def plot_graph(G, pos, filename):
    graph_dir = filename + '_plotgraph.png'
    fig = plt.figure(figsize=(30, 20), dpi=200)
    options = {
        'node_color':'red',
        'node_size':100,
        'alpha':1
    }
    nx.draw_networkx_nodes(G,pos=pos,**options)#draw nodes
    if len(pos) <= 100:
        nx.draw_networkx_edges(G,pos=pos,alpha=0.33,width=0.75)
    else:
        nx.draw_networkx_edges(G,pos=pos,alpha=0.5,width=0.5)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Add padding
    fig.savefig(graph_dir)

In [ ]:
def plot_path(G, pos, path, plot_name):
    fig = plt.figure(figsize=(30, 20), dpi=200)
    H = G.copy()
    
    # Draw closest edges on each node only
    if len(pos) <= 150:
        nx.draw_networkx_edges(H, pos, edge_color="black", width=0.5, alpha=0.1)
    else:
        nx.draw_networkx_edges(H, pos, edge_color="black", width=0.5, alpha=0.5)
    # Draw the route
    nx.draw_networkx(
        G,
        pos,
        node_color = "midnightblue",
        font_color="whitesmoke",
        with_labels=True,
        edgelist=path,
        edge_color="red",
        node_size=400,
        width=3,
    )
    
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Add padding
    plot_name = plot_name + ".png"
    fig.savefig(plot_name)

In [ ]:
# We create the graph for the first (an unique) file in the folder... 
# ... (with .txt extension)
G = create_graph(tsp_files[0])
print(G)
# for this graph we just have the edges between nodes...
# there is not any position, so we decide a possible position
# for visualization
pos = nx.spring_layout(G, seed=7)
plot_graph(G,pos, "bitcoins_plot")

# Here the MSP is solved with Kruskal algorithm
# Other options are ‘prim’, or ‘boruvka’. The default is ‘kruskal’.
##################################################
T = nx.minimum_spanning_tree(G, weight='weight')
##################################################

edge_list = T.edges()
print(T)
MST_dim = 0
for edge in T.edges():
    u, v = edge
    w = float(T[u][v]["weigth"])
    MST_dim = MST_dim + w
print(MST_dim)

plot_path(G, pos, edge_list, "bitcoins_path")
